In [1]:
from ranx import Qrels, Run, evaluate
import bm25s
import Stemmer
from datasets import load_from_disk
from datasets import Dataset, DatasetDict
import numpy as np
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict
import random
import os
from src.utils.project_dirs import get_hfdata_dir
from src.utils.project_dirs import processed_data_dir

In [22]:
def check_monotonicity(df, column='timestamp', group_by=None):
    """
    Checks if a column is monotonically increasing globally and within groups.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str, optional): The column to check for monotonicity. Defaults to 'timestamp'.
        group_by (str, optional): The column to group by. If None, checks global monotonicity only.
            Defaults to None.

    Returns:
        dict: A dictionary containing the results:
            - 'global_monotonic': True if the column is monotonically increasing globally, False otherwise.
            - 'grouped_monotonic': True if the column is monotonically increasing within all groups,
              False if it is not monotonically increasing in at least one group, None if no grouping is performed.
            - 'first_violating_group': The first group where monotonicity is violated, None if no violation.
    """
    results = {}
    results['global_monotonic'] = df[column].is_monotonic_increasing
    results['first_violating_group'] = None # Initialize

    if group_by:
        group_monotonicity = df.groupby(group_by, observed=False)[column].apply(lambda x: x.is_monotonic_increasing)
        results['grouped_monotonic'] = group_monotonicity.all()

        # Find the first group where monotonicity is violated
        violating_groups = group_monotonicity[~group_monotonicity]
        if not violating_groups.empty:
            results['first_violating_group'] = violating_groups.index[0]  # Get the first violating group
        elif results['global_monotonic'] is False:
            results['first_violating_group'] = None
    else:
        results['grouped_monotonic'] = None

    return results

def find_first_violating_user(df, timestamp_column='timestamp', user_id_column='user_id'):
    """
    Finds the first user ID for which the timestamp column is not monotonically increasing.

    Args:
        df (pd.DataFrame): The input DataFrame.
        timestamp_column (str, optional): The name of the timestamp column. Defaults to 'timestamp'.
        user_id_column (str, optional): The name of the user ID column. Defaults to 'user_id'.

    Returns:
        int or None: The first user ID where the timestamp is not monotonically increasing,
                      or None if the timestamp is monotonically increasing for all users.
    """
    monotonicity_check = check_monotonicity(df, column=timestamp_column, group_by=user_id_column)
    if not monotonicity_check['grouped_monotonic']:
        return monotonicity_check['first_violating_group']
    else:
        return None

In [18]:
dataset_name = 'ml1m'

df_ui = pd.read_json(str(processed_data_dir(dataset_name) / 'df_ui.json'), orient='records', lines=True)
display(df_ui.shape, df_ui.head())
print(df_ui.dtypes)

check_monotonicity(df_ui, group_by='user_id')
find_first_violating_user(df_ui)


(1000209, 6)

,user_id,movie_id,rating,timestamp,title,genre
0,1,3186,4,2000-12-31 22:00:19,"Girl, Interrupted (1999)",Drama
1,1,1270,5,2000-12-31 22:00:55,Back to the Future (1985),"Comedy,Sci-Fi"
2,1,1721,4,2000-12-31 22:00:55,Titanic (1997),"Drama,Romance"
3,1,1022,5,2000-12-31 22:00:55,Cinderella (1950),"Animation,Children's,Musical"
4,1,2340,3,2000-12-31 22:01:43,Meet Joe Black (1998),Romance


user_id               int64
movie_id              int64
rating                int64
timestamp    datetime64[ns]
title                object
genre                object
dtype: object


20

In [19]:
df_ui[df_ui['user_id'] == 20]['timestamp']

202723   2001-12-29 23:37:51
202724   2001-12-29 23:37:51
202725   2001-12-29 23:37:51
202726   2001-12-29 23:37:51
202727   2001-12-29 23:38:35
202728   2001-12-29 23:38:35
202729   2001-12-29 23:38:36
202730   2001-12-29 23:39:41
202731   2001-12-29 23:39:41
202732   2001-12-29 23:39:41
202733   2001-12-29 23:40:27
202734   2000-12-30 02:25:06
202735   2000-12-30 02:26:21
202736   2000-12-30 02:28:26
202737   2000-12-30 02:29:04
202738   2000-12-30 02:29:15
202739   2000-12-30 02:29:29
202740   2000-12-30 02:29:29
202741   2000-12-30 02:31:48
202742   2000-12-30 02:31:48
202743   2000-12-30 02:31:48
202744   2000-12-30 02:31:48
202745   2000-12-30 02:31:48
202746   2000-12-30 02:32:56
Name: timestamp, dtype: datetime64[ns]

In [9]:
dataset_name = 'ml100k'

df_ui = pd.read_json(str(processed_data_dir(dataset_name) / 'df_ui.json'), orient='records', lines=True)
display(df_ui.shape, df_ui.head())

df_ui.dtypes

check_monotonicity(df_ui, group_by='user_id')


(100000, 6)

,user_id,movie_id,rating,timestamp,title,genre
0,1,168,5,1997-09-22 21:57:58,Monty Python and the Holy Grail (1974),Comedy
1,1,172,5,1997-09-22 21:57:58,"Empire Strikes Back, The (1980)","Action,Adventure,Drama,Romance,Sci-Fi,War"
2,1,165,5,1997-09-22 21:58:38,Jean de Florette (1986),Drama
3,1,156,4,1997-09-22 21:59:16,Reservoir Dogs (1992),"Crime,Thriller"
4,1,196,5,1997-09-22 22:01:17,Dead Poets Society (1989),Drama


{'global_monotonic': False,
 'first_violating_group': None,
 'grouped_monotonic': True}

In [24]:
dataset_name = 'beauty'

df_ui = pd.read_json(str(processed_data_dir(dataset_name) / 'df_ui.json'), orient='records', lines=True)
display(df_ui.shape, df_ui.head())

df_ui.dtypes

check_monotonicity(df_ui, group_by='reviewerID', column='unixReviewTime')


(198371, 5)

,reviewerID,asin,unixReviewTime,overall,title
0,A00414041RD0BXM6WK0GX,B007IY97U0,1405296000,3,63cm Long Zipper Beige+pink Wavy Cosplay Hair ...
1,A00414041RD0BXM6WK0GX,B00870XLDS,1405296000,2,MapofBeauty Long Wave Curly Hair Wig Full Wig ...
2,A00414041RD0BXM6WK0GX,B008MIRO88,1405296000,1,MapofBeauty Cosplay Costume Long Curly Hair Wi...
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,1405296000,3,32&quot; 80cm Long Hair Heat Resistant Spiral ...
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,1405296000,5,MapofBeauty 28&quot; 70cm Long Curly Hair Ends...


{'global_monotonic': False,
 'first_violating_group': None,
 'grouped_monotonic': True}

In [25]:
dataset_name = 'toys'

df_ui = pd.read_json(str(processed_data_dir(dataset_name) / 'df_ui.json'), orient='records', lines=True)
display(df_ui.shape, df_ui.head())

df_ui.dtypes

check_monotonicity(df_ui, group_by='reviewerID', column='unixReviewTime')


(166757, 5)

,reviewerID,asin,unixReviewTime,overall,title
0,A012468118FTQAINEI0OQ,B001HA9JOA,1356220800,5,Chutes and Ladders Super Hero Squad
1,A012468118FTQAINEI0OQ,B00804BCO6,1356220800,5,LeapFrog Letter Factory Phonics
2,A012468118FTQAINEI0OQ,B00005BZM6,1360108800,5,Crayola Art Smock-
3,A012468118FTQAINEI0OQ,B002BY2BVE,1360108800,5,"Crayola Giant Fingerpaint Paper, (99-3405)"
4,A012468118FTQAINEI0OQ,B003S7EYZY,1360108800,5,Gund Philbin 13&quot; Bear


{'global_monotonic': False,
 'first_violating_group': None,
 'grouped_monotonic': True}

In [26]:
dataset_name = 'sports'

df_ui = pd.read_json(str(processed_data_dir(dataset_name) / 'df_ui.json'), orient='records', lines=True)
display(df_ui.shape, df_ui.head())

df_ui.dtypes

check_monotonicity(df_ui, group_by='reviewerID', column='unixReviewTime')


(295091, 5)

,reviewerID,asin,unixReviewTime,overall,title
0,A00046902LP5YSDV0VVNF,B0010O748Q,1353628800,5,Magnesium Fire Starter
1,A00046902LP5YSDV0VVNF,B00178CS4K,1353628800,5,Survivor HK-106320 Outdoor Fixed Blade Knife 7...
2,A00046902LP5YSDV0VVNF,B001THHXA8,1353628800,5,Big Bohica Kukhri Machete
3,A00046902LP5YSDV0VVNF,B004U8CP88,1353628800,2,Ultralight Backpacking Canister Camp Stove wit...
4,A00046902LP5YSDV0VVNF,B007BO931U,1353628800,4,Tac Forece TF-710 Series Assisted Opening Fold...


{'global_monotonic': False,
 'first_violating_group': None,
 'grouped_monotonic': True}

In [9]:
import pandas as pd
from src.utils.project_dirs import processed_data_dir

def get_seqlen_stats(dataset_name, low_seqlen, high_seqlen, user_col='user_id', item_col='item_id'):
    df_ui = pd.read_json(str(processed_data_dir(dataset_name) / 'df_ui.json'), orient='records', lines=True)
    usc = df_ui[user_col].value_counts() # usser seqlen count, will count number of times each user has rated any item, could be repeated items
    print("#############")
    
    print("mean: ", usc.mean(), "std: ", usc.std())
    print(dataset_name,(usc <= low_seqlen).sum() / len(usc), (usc > high_seqlen).sum() / len(usc))

    cold_start_users = set(usc[usc <= low_seqlen].index.tolist())
    regular_users = set(usc[(usc > low_seqlen) & (usc <= high_seqlen)].index.tolist())
    power_users = set(usc[usc > high_seqlen].index.tolist())


    print("Total number of users: ", len(usc))
    print(f"Number of cold-start users, pcent usplit: {len(cold_start_users)},{100*len(cold_start_users)/len(usc)}")
    print(f"Number of regular users, pcent usplit: {len(regular_users)},{100*len(regular_users)/len(usc)}")
    print(f"Number of power users, pcent usplit: {len(power_users)},{100*len(power_users)/len(usc)}")
    print(f"Sanity check: {len(usc)==len(cold_start_users)+len(regular_users)+len(power_users)}")

    # find datasplit, basically the sum of the number of users in each split
    print("Total number of interactions: ", len(df_ui))
    cs_users_itemseqsum = df_ui[df_ui[user_col].isin(cold_start_users)].shape[0]
    rs_users_itemseqsum = df_ui[df_ui[user_col].isin(regular_users)].shape[0]
    ps_users_itemseqsum = df_ui[df_ui[user_col].isin(power_users)].shape[0]
    print("Cold start user interactions, pcent dsplit: ", cs_users_itemseqsum, 100*cs_users_itemseqsum/len(df_ui))
    print("Regular user interactions, pcent dsplit: ", rs_users_itemseqsum, 100*rs_users_itemseqsum/len(df_ui))
    print("Power user interactions, pcent dsplit: ", ps_users_itemseqsum, 100*ps_users_itemseqsum/len(df_ui))
    print(f"Sanity check: {cs_users_itemseqsum+rs_users_itemseqsum+ps_users_itemseqsum==len(df_ui)}") # total number of interactions should match up

    return cold_start_users, regular_users, power_users

dataset_name = 'beauty'
cold_beauty, regular_beauty, power_beauty = get_seqlen_stats(dataset_name, low_seqlen = 5, high_seqlen=14, user_col =  'reviewerID', item_col = 'asin')

dataset_name = 'sports'
cold_sports, regular_sports, power_sports = get_seqlen_stats(dataset_name, low_seqlen = 5, high_seqlen=13, user_col =  'reviewerID', item_col = 'asin')

dataset_name = 'toys'
cold_toys, regular_toys, power_toys = get_seqlen_stats(dataset_name, low_seqlen = 5, high_seqlen=13, user_col =  'reviewerID', item_col = 'asin')


from src.utils.project_dirs import get_peruser_metric_dataset_modelname_encoder
cols = ['recall@5', 'ndcg@5']
dataset = 'beauty'
encoder_name = 'sentence-transformers/all-MiniLM-L6-v2'

model_name = 'llama-1b'
dir = get_peruser_metric_dataset_modelname_encoder(dataset=dataset, model_name=model_name, encoder_name=encoder_name)

beauty_pum_1b = pd.read_json(str(dir / 'beauty_test_llama-1b.jsonl'), orient='records', lines=True)

cold_start_beauty_1b_metrics = beauty_pum_1b[beauty_pum_1b['reviewer_id'].isin(cold_beauty)]
regular_beauty_1b_metrics = beauty_pum_1b[beauty_pum_1b['reviewer_id'].isin(regular_beauty)]
power_beauty_1b_metrics = beauty_pum_1b[beauty_pum_1b['reviewer_id'].isin(power_beauty)]

print(cold_start_beauty_1b_metrics[cols].mean())



#############
mean:  8.870500380092116 std:  8.160898111531392
beauty 0.3210660465948218 0.09779546572463443
Total number of users:  22363
Number of cold-start users, pcent usplit: 7180,32.10660465948218
Number of regular users, pcent usplit: 12996,58.113848768054375
Number of power users, pcent usplit: 2187,9.779546572463444
Sanity check: True
Total number of interactions:  198371
Cold start user interactions, pcent dsplit:  35871 18.08278427794385
Regular user interactions, pcent dsplit:  104647 52.753174607175445
Power user interactions, pcent dsplit:  57853 29.164041114880703
Sanity check: True
#############
mean:  8.289771609967133 std:  6.0523199381244845
sports 0.32598252661741156 0.09891282973284266
Total number of users:  35597
Number of cold-start users, pcent usplit: 11604,32.59825266174116
Number of regular users, pcent usplit: 20472,57.510464364974574
Number of power users, pcent usplit: 3521,9.891282973284266
Sanity check: True
Total number of interactions:  295091
Cold s

In [25]:
from src.utils.project_dirs import get_peruser_metric_dataset_modelname_encoder
cols = ['recall@5', 'ndcg@5']
dataset = 'beauty'
encoder_name = 'sentence-transformers/all-MiniLM-L6-v2'

model_name = 'llama-1b'
dir = get_peruser_metric_dataset_modelname_encoder(dataset=dataset, model_name=model_name, encoder_name=encoder_name)

beauty_pum_1b = pd.read_json(str(dir / 'beauty_test_llama-1b.jsonl'), orient='records', lines=True)

cold_start_beauty_1b_metrics = beauty_pum_1b[beauty_pum_1b['reviewer_id'].isin(cold_beauty)]
regular_beauty_1b_metrics = beauty_pum_1b[beauty_pum_1b['reviewer_id'].isin(regular_beauty)]
power_beauty_1b_metrics = beauty_pum_1b[beauty_pum_1b['reviewer_id'].isin(power_beauty)]

# print(cold_start_beauty_1b_metrics[cols].mean())
# print(regular_beauty_1b_metrics[cols].mean())
# print(power_beauty_1b_metrics[cols].mean())

model_name = 'llama-8b'
dir = get_peruser_metric_dataset_modelname_encoder(dataset=dataset, model_name=model_name, encoder_name=encoder_name)
beauty_pum_8b = pd.read_json(str(dir / 'beauty_test_llama-8b.jsonl'), orient='records', lines=True)

cold_start_beauty_8b_metrics = beauty_pum_8b[beauty_pum_8b['reviewer_id'].isin(cold_beauty)]
regular_beauty_8b_metrics = beauty_pum_8b[beauty_pum_8b['reviewer_id'].isin(regular_beauty)]
power_beauty_8b_metrics = beauty_pum_8b[beauty_pum_8b['reviewer_id'].isin(power_beauty)]

csb8 = cold_start_beauty_8b_metrics[cols].mean()
rb8 = regular_beauty_8b_metrics[cols].mean()
pb8 = power_beauty_8b_metrics[cols].mean()

print(csb8)
print(rb8)
print(pb8)
print((csb8*len(cold_beauty) + rb8*len(regular_beauty) + pb8*len(power_beauty))/len(beauty_pum_8b)) # just a sanity check
print(beauty_pum_8b[cols].mean())






recall@5    0.061699
ndcg@5      0.039863
dtype: float64
recall@5    0.063943
ndcg@5      0.040730
dtype: float64
recall@5    0.112940
ndcg@5      0.071061
dtype: float64
recall@5    0.068014
ndcg@5      0.043418
dtype: float64
recall@5    0.068014
ndcg@5      0.043418
dtype: float64


In [20]:
power_beauty_1b_metrics

,reviewer_id,recall@5,ndcg@5,mrr
9,A03364251DGXSGA9PSR99,0,0.0,0.0
29,A099766128UI0NCS98N1E,0,0.0,0.0
35,A100WO06OQR8BQ,0,0.0,0.0
37,A100ZQDV7L8PVV,0,0.0,0.0
47,A103979529MRJY0U56QI4,0,0.0,0.0
...,...,...,...,...
22280,AZFHSPEZUPGD2,0,0.0,0.0
22288,AZIDXJ9F7A83Z,0,0.0,0.0
22307,AZMY6E8B52L2T,0,0.0,0.0
22312,AZOO27I3Z0IQO,0,0.0,0.0


In [18]:
cold_start_user_metrics.shape

(7180, 4)

In [16]:
beauty_pum_1b

,reviewer_id,recall@5,ndcg@5,mrr
0,A00414041RD0BXM6WK0GX,0,0.0,0.0
1,A00473363TJ8YSZ3YAGG9,0,0.0,0.0
2,A00700212KB3K0MVESPIY,0,0.0,0.0
3,A0078719IR14X3NNUG0F,0,0.0,0.0
4,A01198201H0E3GHV2Z17I,0,0.0,0.0
...,...,...,...,...
22358,AZZNK89PXD006,0,0.0,0.0
22359,AZZQXL8VDCFTV,0,0.0,0.0
22360,AZZT1ERHBSNQ8,0,0.0,0.0
22361,AZZU6NXB8YJN9,0,0.0,0.0


In [81]:
from src.utils.project_dirs import get_peruser_metric_dataset_modelname_encoder
dir = get_peruser_metric_dataset_modelname_encoder(dataset='ml100k', model_name='llama-8b', encoder_name='bm25s')
pum = pd.read_json(str(dir / 'ml100k_test_llama-8b.jsonl'), orient='records', lines=True)

pum.describe()


,user_id,recall@5,ndcg@5,mrr
count,943.000000,943.000000,943.000000,943.000000
mean,472.000000,0.060445,0.036588,0.028844
std,272.364951,0.238437,0.154952,0.135768
min,1.000000,0.000000,0.000000,0.000000
25%,236.500000,0.000000,0.000000,0.000000
50%,472.000000,0.000000,0.000000,0.000000
75%,707.500000,0.000000,0.000000,0.000000
max,943.000000,1.000000,1.000000,1.000000


In [82]:
from src.utils.project_dirs import get_peruser_metric_dataset_modelname_encoder
dir = get_peruser_metric_dataset_modelname_encoder(dataset='ml100k', model_name='llama-8b', encoder_name='sentence-transformers/all-MiniLM-L6-v2')
pum = pd.read_json(str(dir / 'ml100k_test_llama-8b.jsonl'), orient='records', lines=True)

pum.describe()


,user_id,recall@5,ndcg@5,mrr
count,943.000000,943.000000,943.000000,943.000000
mean,472.000000,0.060445,0.035945,0.027943
std,272.364951,0.238437,0.151304,0.130244
min,1.000000,0.000000,0.000000,0.000000
25%,236.500000,0.000000,0.000000,0.000000
50%,472.000000,0.000000,0.000000,0.000000
75%,707.500000,0.000000,0.000000,0.000000
max,943.000000,1.000000,1.000000,1.000000
